# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        # print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer1.py


In [3]:
%%writefile test.dat

1	hello world
2	hello again
42	Привет мир!
3	Another example with heLLo
4	For The Horde!
5	Hello again

Writing test.dat


In [4]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py

again	2
another	1
example	1
for	1
hello	4
horde	1
the	1
with	1
world	1
мир	1
привет	1


## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [5]:
%%writefile mapper2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, cnt = line.strip().split('\t', 1)
        cnt = int(cnt)
    except ValueError as e:
        continue    
    print "%d\t%s" % (cnt, word)

Writing mapper2.py


In [6]:
%%writefile reducer2.py

import sys

for line in sys.stdin:
    try:
        cnt, word = line.strip().split('\t', 1)
        cnt = int(cnt)
    except ValueError as e:
        continue
    print "%s\t%d" % (word, cnt)

Writing reducer2.py


In [7]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py | python2 ./mapper2.py | sort -r | python2 ./reducer2.py

hello	4
again	2
привет	1
мир	1
world	1
with	1
the	1
horde	1
for	1
example	1
another	1


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [8]:
%%bash

DIR1="wordCount_"$(date +"%s%6N")
DIR2="wordsRaiting_"$(date +"%s%6N")

hdfs dfs -rm -r -skipTrash "wordCount_"* > /dev/null
hdfs dfs -rm -r -skipTrash "wordsRaiting_"* > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=8 \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${DIR1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordsRaiting" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${DIR1} \
    -output ${DIR2} > /dev/null

hdfs dfs -cat ${DIR2}/part-00000 | sed -n '7p;8q'

is	126420


rm: `wordCount_*': No such file or directory
rm: `wordsRaiting_*': No such file or directory
19/04/15 03:30:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/15 03:30:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/04/15 03:30:41 INFO mapred.FileInputFormat: Total input files to process : 1
19/04/15 03:30:41 INFO mapreduce.JobSubmitter: number of splits:2
19/04/15 03:30:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1555291247429_0001
19/04/15 03:30:41 INFO impl.YarnClientImpl: Submitted application application_1555291247429_0001
19/04/15 03:30:41 INFO mapreduce.Job: The url to track the job: http://67c08c68fb41:8088/proxy/application_1555291247429_0001/
19/04/15 03:30:41 INFO mapreduce.Job: Running job: job_1555291247429_0001
19/04/15 03:30:48 INFO mapreduce.Job: Job job_1555291247429_0001 running in uber mode : false
19/04/15 03:30:48 INFO mapreduce.Job:  map 0% reduce 0%
19/04/15 03:31:05 INFO mapreduce.Job:  map 4